In [1]:
import unidecode 
import numpy as np

In [2]:
def get_soundex_word(palavra):
    palavra = unidecode.unidecode(palavra)
    for index, letra in enumerate(palavra.upper()):
        if index == 0:
            soundex = letra
            letra_anterior = letra        
            continue

        if letra in "AEIOUHW":
            letra_anterior = letra        
            continue    
        
        dict_cod = {"BFPV": "1",
                    "CÇGJKQSXZ": "2",
                    "DT": "3",
                    "L": "4",
                    "MN": "5",
                    "R": "6"}
        
        
        # assume o valor do codigo correspondente a letra no dicionario dict_cod ou o valor 0
        cod = '0' if letra not in "".join([*dict_cod]) else [dict_cod[key] for key in dict_cod.keys() if letra in key][0]
        
        if cod == '0':
            continue

        # se a letra anterior estiver em "hw" ela não é contabilizada e se a letra repetir também não conta 2 vezes
        if letra_anterior in "HW" and soundex[-1] == cod:
            letra_anterior = letra
            continue

        # se a letra anterior estiver em "aeiou" ela não é contabilizada mas se a letra repetir ela conta 2 vezes
        # ou se a letra anterior for diferente da letra atual, também irá adicionar o novo código da letra
        elif letra_anterior in "AEIOU" or soundex[-1] != cod:            
            letra_anterior = letra
            soundex += cod
            if len(soundex) == 4:
                return soundex
            continue

    
    return soundex.ljust(4, '0')
            
    
def get_soundex(string, separar_palavras=False):
    soundex = []
    
    # se a entrada for uma lista
    if isinstance(string, list):
        for texto in string:
            if separar_palavras:
                for palavra in texto.split():
                    palavra = palavra.strip()
                    palavra = palavra.translate(str.maketrans('', '', "-!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~"))
                    soundex.append(get_soundex_word(palavra))
            else:
                texto = texto.translate(str.maketrans('', '', "-!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~"))
                soundex.append(get_soundex_word(texto.replace(" ", "")))
        return soundex
    
    if separar_palavras:
        string = string.split()
        for palavra in string:
            palavra = palavra.translate(str.maketrans('', '', "-!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~"))
            soundex.append(get_soundex_word(palavra))
        return soundex
    else:
        string = string.translate(str.maketrans('', '', "-!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~"))
        soundex.append(get_soundex_word(string.replace(" ", "")))
        return soundex

In [3]:
def edit_distance(w1, w2):
    cols = len(w1)+ 1
    rows = len(w2) + 1
    matrix = np.zeros([rows,cols])
    
    for i in range(cols):
        matrix[0][i] = i
    for j in range(rows):
        matrix[j][0] = j
    
    for i in range(1, rows):
        for j in range(1, cols):
            cost_rep = matrix[i-1][j-1]
            cost_del = matrix[i-1][j]
            cost_add = matrix[i][j-1]
            # se as letras não coincidirem
            if min(cost_rep, cost_del, cost_add) == cost_rep and w1[j-1] == w2[i-1]:            
                cost = cost_rep
            else:
                cost = min(cost_rep, cost_del, cost_add) + 1
            
            matrix[i][j] = cost
                
    return matrix[-1,-1]

In [4]:
with open('Lista-de-Palavras.txt', encoding='utf-8') as f:
    palavras = f.read().splitlines()

print(f"Palavras antes da limpeza: {len(palavras)}")

palavras = [palavra.lower() for palavra in palavras if not any(letra in "-!\"#$%&'()*+,./:;<=>?@[\\]^_`{|}~" for letra in palavra)]
print(f"Palavras depos da limpeza: {len(palavras)}")

Palavras antes da limpeza: 29858
Palavras depos da limpeza: 28637


In [5]:
# dicionário com as chaves sendo as palavras e os valores sendo o soundex da palavra
palavras_codigo = {palavra: get_soundex(palavra)[0] for palavra in palavras}
palavras_codigo_np = np.array(list(palavras_codigo.items()))

In [6]:
# retorna as possíveis correções para a palavra com erro de ortografia
def get_escrita_correta(palavra_digitada):
    palavra_digitada = palavra_digitada.lower()
    cod = get_soundex(palavra_digitada)[0]
    possiveis_correcoes = {}
    
    for palavra_codigo in palavras_codigo_np:
        if edit_distance(palavra_codigo[1], cod) < 3 and edit_distance(palavra_codigo[0], palavra_digitada) < 3:
            possiveis_correcoes[palavra_codigo[0]] = edit_distance(palavra_codigo[0], palavra_digitada)
    
    # ordena as palavras da mais provável pra menos provavel
    possiveis_correcoes = {k: v for k, v in sorted(possiveis_correcoes.items(), key=lambda item: item[1])}
    # obtém apenas as 5 palavras mais prováveis
    possiveis_correcoes = {k: possiveis_correcoes[k] for k in list(possiveis_correcoes)[:5]}
    return possiveis_correcoes

In [7]:
# retorna dicionário com as chaves sendo as palavras com erros de ortografia e os valroes são outros dicionários
# com as chaves sendo as possíveis correcoes e os valores a % de conficança que aquela é a palavra correta
def get_correcao(*palavras, normalizar_acentuacao=True):
    correcoes_total = {}
    for palavra in palavras:
        correcoes = {}
        if normalizar_acentuacao:
            palavra_normalizada = unidecode.unidecode(palavra)
            possiveis_correcoes = get_escrita_correta(palavra_normalizada)
        else:
            possiveis_correcoes = get_escrita_correta(palavra)
        sum_dist = sum(possiveis_correcoes.values())
        sum_dist_para_divisao = 0
        for pos_cor, dist in possiveis_correcoes.items():
            if dist == 0:
                correcoes[pos_cor] = 1
                break
            sum_dist_para_divisao += (dist/sum_dist)**-1
        else:
            for possivel_correcao, dist in possiveis_correcoes.items():
                pct_chance = round((dist/sum_dist)**-1 / sum_dist_para_divisao, 3)
                correcoes[possivel_correcao] = pct_chance
        correcoes_total[palavra] = correcoes
    return correcoes_total

In [8]:
# verifica as correções para as palavras que entram na função
corrigindo = get_correcao('onstituir', "comumicacao", 'comforme', 'voxe')
for key, value in corrigindo.items():
    print(f'Palavra digitada de forma errada: "{key}"')
    for k, v in value.items():
        print(f'Possivel correcao: "{k}". '.ljust(35, ' '), f'Precisão: {round(v*100, 2)}%')
    print('-'*30)

Palavra digitada de forma errada: "onstituir"
Possivel correcao: "constituir".    Precisão: 50.0%
Possivel correcao: "instituir".     Precisão: 50.0%
------------------------------
Palavra digitada de forma errada: "comumicacao"
Possivel correcao: "comunicacao".   Precisão: 66.7%
Possivel correcao: "complicacao".   Precisão: 33.3%
------------------------------
Palavra digitada de forma errada: "comforme"
Possivel correcao: "conforme".      Precisão: 100.0%
------------------------------
Palavra digitada de forma errada: "voxe"
Possivel correcao: "boxe".          Precisão: 25.0%
Possivel correcao: "voce".          Precisão: 25.0%
Possivel correcao: "vox".           Precisão: 25.0%
Possivel correcao: "axe".           Precisão: 12.5%
Possivel correcao: "bode".          Precisão: 12.5%
------------------------------
